### Script that converts NASA 9km standard mapped image (4320*2160) to lower resolution equal area map.
### See rioxarray and below links

####### https://stackoverflow.com/questions/70997075/using-xarray-interp-to-reproject-a-dataarray

####### Haven't tried but look at https://gis.stackexchange.com/questions/373649/is-it-possible-to-use-esri54009-world-mollweid-proj-with-rasterio

In [ ]:
import xarray as xr
import numpy as np
import rioxarray 
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from pyproj import Transformer
import os 
import glob
import xarray as xr

## Reduce resolution and reproject to equal area Mollweide projection (ESRI:54009)

In [ ]:
# https://zenodo.org/records/12586956 or equivalent
ds = xr.open_dataset('/media/gsilsbe/nasa_npp/cafe/archival/cafe_npp.nc', decode_coords="all").npp
ds = ds.rio.write_crs('EPSG:4326')
ds = ds.rio.set_spatial_dims("lon","lat",inplace=True)
ds = ds.rio.write_nodata(np.nan, inplace=True)
dst = ds.rio.reproject('ESRI:54009', resolution=(50000, 50000), resampling=Resampling.bilinear) 
dst

## EOF Analysis

In [ ]:
ipc = 6 # Save these many principal components

da = dst.stack(z=('y','x')).values

# Keep only ocean data 
mask = np.abs(np.sum(da, axis=0))
mask = np.where(mask >= 0 , 1, 0)
ocean_z = np.argwhere(mask.flatten()==1).flatten()
da = da[:,ocean_z]

skpca = decomposition.PCA()
skpca.fit(da)       

# Now saves the (fitted) PCA object for reuse in operations
joblib.dump(skpca, '../EOF.pkl', compress=9)

# Save expplained variance ratio
PCdf_evr = pd.DataFrame(skpca.explained_variance_ratio_[0:ipc])

# The Principal Components (PCs) are obtained by using the transform method of the pca object (skpca)
PCs = skpca.transform(da)
PCs = PCs[:,:ipc]
EOFs = skpca.components_
EOFs = EOFs[:ipc,:]
EOF_recons = np.empty([ipc, len(dst.stack(z=('y','x')).z)])
EOF_recons[:] = np.NaN

# Scale the Principal Components
scaler_PCs = preprocessing.StandardScaler()
scaler_PCs.fit(PCs)
PCs_std = scaler_PCs.transform(PCs)
joblib.dump(scaler_PCs, './scaler_PCs.pkl')

PCdf = pd.DataFrame(PCs_std, index = dst['time'], columns = ["EOF%s" % (x) for x in range(1, PCs_std.shape[1] +1)])

for i in range(ipc): 
    EOF_recons[i,ocean_z] = EOFs[i,:]

# Convert back to xarray 
EOF = xr.Dataset(
    data_vars=dict(
        EOF_pcs=(["pc", "z"], EOF_recons)
    ),
    coords={'pc':np.arange(ipc), 'z': dst.stack(z=('y','x')).z})
EOF = EOF.unstack()        
        
